#Transformation from bronze to silver

We access to the storage account and set the paths to the files

In [0]:
###Establish account key G2
spark.conf.set("fs.azure.account.key.tfmgendos.dfs.core.windows.net","QT/ULR9KopMmUZmz55avGjsJf/cuoAdFd/HUT3qDxs5pLllRhvXwyQEsqn8b1RKn91hv7lTuhb9K+AStoUFp+A==")

basepath="abfss://tfm@tfmgendos.dfs.core.windows.net/"

dbutils.fs.ls(basepath)


Out[1]: [FileInfo(path='abfss://tfm@tfmgendos.dfs.core.windows.net/bronze/', name='bronze/', size=0, modificationTime=1662129140000),
 FileInfo(path='abfss://tfm@tfmgendos.dfs.core.windows.net/gold/', name='gold/', size=0, modificationTime=1662129150000),
 FileInfo(path='abfss://tfm@tfmgendos.dfs.core.windows.net/landing/', name='landing/', size=0, modificationTime=1662129135000),
 FileInfo(path='abfss://tfm@tfmgendos.dfs.core.windows.net/silver/', name='silver/', size=0, modificationTime=1662129145000)]

In [0]:



# autoloader table and checkpoint paths

#bronze path
WEOByCountryGroup_Bronce = basepath + "bronze/fmi/WEOByCountryGroup/"
bronzeCheckpoint = basepath + "bronze/fmi/WEOByCountryGroup/checkpoint/"
bronzeSchema = basepath + "bronze/fmi/WEOByCountryGroup/schema/"

# Landing Zone path
landingZoneLocation = basepath + "landing/fmi/weobycountrygroup/"

#Silver path
actualAndForecastCommoditiesPrices_Silver = basepath + "silver/fmi/WEOByCountryGroup/actualAndForecastCommoditiesPrices"






In [0]:

#Load Bronze table
dfBronze = spark.read.format("delta").load(WEOByCountryGroup_Bronce)

#display(dfBronze)

In [0]:
dfBronze.columns

Out[4]: ['WEO_Country_Group_Code',
 'WEO_Subject_Code',
 'Country_Group_Name',
 'Subject_Descriptor',
 'Subject_Notes',
 'Units',
 'Scale',
 'Country/Series-specific_Notes',
 '1980',
 '1981',
 '1982',
 '1983',
 '1984',
 '1985',
 '1986',
 '1987',
 '1988',
 '1989',
 '1990',
 '1991',
 '1992',
 '1993',
 '1994',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '2022',
 '2023',
 '2024',
 '2025',
 '2026',
 '2027',
 'Estimates_Start_After',
 '_rescued_data']

We prepare the data to the use case we are aiming to

In [0]:
#Using SQL col() function
from pyspark.sql.functions import col 


dfSilver = dfBronze.drop("_rescued_data","WEO_Country_Group_Code","WEO_Subject_Code","Country/Series-specific_Notes", "Subject_Notes", "Scale")


dfSilver.count()

Out[5]: 1665

In [0]:
#filter by specific subject_descriptor value
dfSilver = dfSilver.filter(dfSilver.Subject_Descriptor.startswith("Natural Gas") | dfSilver.Subject_Descriptor.startswith("Crude Oil (petroleum),  Dated Brent, light blend 38 API, fob U.K., US$ per barrel") | dfSilver.Subject_Descriptor.startswith("Oil; Dubai, medium, Fateh 32 API, fob Dubai Crude Oil (petroleum), Dubai Fateh Fateh 32 API, US$ per barrel") | dfSilver.Subject_Descriptor.startswith("Crude Oil (petroleum), West Texas Intermediate 40 API, Midland Texas, US$ per barrel") | dfSilver.Subject_Descriptor.startswith("Aluminum, 99.5% minimum purity, LME spot price, CIF UK ports, US$ per metric tonne") | dfSilver.Subject_Descriptor.startswith("Iron Ore, China import Iron Ore Fines 62% FE spot (CFR Tianjin port) US$ per metric ton"))

In [0]:
display(dfSilver)

Country_Group_Name,Subject_Descriptor,Units,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,Estimates_Start_After
World,"Crude Oil (petroleum), Dated Brent, light blend 38 API, fob U.K., US$ per barrel",U.S. dollars,37.892,36.676,33.418,29.783,28.743,27.611,14.433,18.436,14.975,18.248,23.711,19.982,19.414,16.998,15.826,17.056,20.453,19.116,12.719,17.699,28.308,24.412,24.998,28.845,38.281,54.433,65.376,72.731,97.569,61.865,79.635,111.082,111.960,108.844,98.943,52.399,44.048,54.401,71.071,64.032,42.298,70.443,110.815,96.092,87.238,81.363,77.583,74.923,null
World,"Oil; Dubai, medium, Fateh 32 API, fob Dubai Crude Oil (petroleum), Dubai Fateh Fateh 32 API, US$ per barrel",U.S. dollars,35.846,34.294,31.757,28.468,27.509,26.508,13.056,16.964,13.360,15.781,20.730,16.607,17.138,14.906,14.832,16.128,18.537,18.102,12.090,17.076,26.085,22.710,23.734,26.732,33.440,49.192,61.412,68.405,93.713,61.729,78.064,106.027,108.919,105.430,96.664,51.232,41.236,53.116,69.130,63.147,42.149,68.781,105.802,92.184,84.418,79.380,76.002,74.720,null
World,"Crude Oil (petroleum), West Texas Intermediate 40 API, Midland Texas, US$ per barrel",U.S. dollars,37.879,36.168,32.670,30.604,29.388,27.992,15.024,19.193,15.972,19.692,24.518,21.512,20.555,18.455,17.183,18.432,22.132,20.585,14.416,19.166,30.315,25.869,26.116,31.133,41.429,56.455,66.036,72.347,99.612,61.702,79.405,95.046,94.143,97.934,93.135,48.748,43.226,50.924,64.785,56.995,39.413,67.978,103.868,89.609,81.021,74.892,70.517,67.789,null
World,"Natural Gas, Russian Natural Gas border price in Germany, US$ per million metric British thermal units of gas",U.S. dollars,n/a,n/a,n/a,n/a,n/a,3.810,3.440,2.130,1.900,1.589,2.035,3.008,2.365,2.598,2.312,2.696,2.750,2.670,2.245,1.807,3.454,3.873,2.666,3.486,3.755,5.915,8.213,8.143,13.138,8.855,8.223,10.597,11.981,11.189,10.463,7.305,4.353,5.744,7.915,4.449,3.179,15.912,48.171,26.127,14.653,10.593,8.922,8.106,null
World,"Natural Gas, Indonesian Liquified Natural Gas in Japan, US$ per million metric British thermal units of liquid",U.S. dollars,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,3.744,3.598,3.285,3.557,3.903,3.761,2.754,3.343,5.235,4.669,4.427,4.981,5.891,7.033,8.049,8.411,11.591,7.491,9.381,15.553,18.145,17.339,16.998,10.958,7.439,7.247,9.795,5.445,4.366,18.600,44.883,27.048,27.048,27.048,27.048,27.048,null
World,"Natural Gas, Natural Gas spot price at the Henry Hub terminal in Louisiana, US$ per million metric British thermal units of gas",U.S. dollars,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,1.451,1.751,2.114,1.915,1.704,2.689,2.460,2.088,2.261,4.308,3.959,3.361,5.507,5.908,8.895,6.740,6.982,8.858,3.946,4.386,4.000,2.752,3.726,4.370,2.614,2.492,2.964,3.155,2.566,2.010,3.850,4.783,3.901,3.418,3.374,3.418,3.515,null
World,"Aluminum, 99.5% minimum purity, LME spot price, CIF UK ports, US$ per metric tonne",U.S. dollars,"1,774.913","1,262.730",991.568,"1,438.436","1,251.611","1,040.728","1,149.709","1,565.096","2,546.520","1,950.714","1,639.500","1,304.020","1,256.275","1,139.931","1,475.630","1,805.020","1,506.794","1,599.294","1,357.572","1,359.989","1,551.498","1,446.747","1,351.057","1,432.817","1,718.496","1,900.506","2,573.063","2,639.861","2,577.916","1,668.489","2,172.994","2,400.643","2,022.795","1,846.677","1,867.421","1,664.681","1,604.182","1,967.655","2,108.474","1,794.488","1,704.096","2,472.952","3,478.582","3,359.750","3,096.250","2,882.333","2,695.250","2,597.500",null
World,"Iron Ore, China import Iron Ore Fines 62% FE spot (CFR Tianjin port) US$ per metric ton",U.S. dollars,12.150,12.150,14.050,12.540,11.310,11.490,11.360,10.940,10.510,12.030,14.050,15.030,14.310,12.580,11.450,12.270,12.970,13.040,13.410,11.930,12.450,12.990,12.680,13.820,16.390,28.110,33.450,36.630,61.565,79.994,146.720,167.790,128.526,135.361,97.387,56.137,58.565,71.134,70.138,

In [0]:

#Keep rows with Country_group_name == "World"
dfSilver = dfSilver.filter(dfSilver.Country_Group_Name == "World")

dfSilver = dfSilver.drop("Country_Group_Name")


In [0]:
dfSilver.printSchema()

root
-- Subject_Descriptor: string (nullable = true)
-- Units: string (nullable = true)
-- 1980: string (nullable = true)
-- 1981: string (nullable = true)
-- 1982: string (nullable = true)
-- 1983: string (nullable = true)
-- 1984: string (nullable = true)
-- 1985: string (nullable = true)
-- 1986: string (nullable = true)
-- 1987: string (nullable = true)
-- 1988: string (nullable = true)
-- 1989: string (nullable = true)
-- 1990: string (nullable = true)
-- 1991: string (nullable = true)
-- 1992: string (nullable = true)
-- 1993: string (nullable = true)
-- 1994: string (nullable = true)
-- 1995: string (nullable = true)
-- 1996: string (nullable = true)
-- 1997: string (nullable = true)
-- 1998: string (nullable = true)
-- 1999: string (nullable = true)
-- 2000: string (nullable = true)
-- 2001: string (nullable = true)
-- 2002: string (nullable = true)
-- 2003: string (nullable = true)
-- 2004: string (nullable = true)
-- 2005: string (nullable = true)
-- 2006: string (nullable = true)
-- 2007: string (nullable = true)
-- 2008: string (nullable = true)
-- 2009: string (nullable = true)
-- 2010: string (nullable = true)
-- 2011: string (nullable = true)
-- 2012: string (nullable = true)
-- 2013: string (nullable = true)
-- 2014: string (nullable = true)
-- 2015: string (nullable = true)
-- 2016: string (nullable = true)
-- 2017: string (nullable = true)
-- 2018: string (nullable = true)
-- 2019: string (nullable = true)
-- 2020: string (nullable = true)
-- 2021: string (nullable = true)
-- 2022: string (nullable = true)
-- 2023: string (nullable = true)
-- 2024: string (nullable = true)
-- 2025: string (nullable = true)
-- 2026: string (nullable = true)
-- 2027: string (nullable = true)
-- Estimates_Start_After: string (nullable = true)

In [0]:
display(dfSilver)

Subject_Descriptor,Units,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,Estimates_Start_After
"Crude Oil (petroleum), Dated Brent, light blend 38 API, fob U.K., US$ per barrel",U.S. dollars,37.892,36.676,33.418,29.783,28.743,27.611,14.433,18.436,14.975,18.248,23.711,19.982,19.414,16.998,15.826,17.056,20.453,19.116,12.719,17.699,28.308,24.412,24.998,28.845,38.281,54.433,65.376,72.731,97.569,61.865,79.635,111.082,111.960,108.844,98.943,52.399,44.048,54.401,71.071,64.032,42.298,70.443,110.815,96.092,87.238,81.363,77.583,74.923,null
"Oil; Dubai, medium, Fateh 32 API, fob Dubai Crude Oil (petroleum), Dubai Fateh Fateh 32 API, US$ per barrel",U.S. dollars,35.846,34.294,31.757,28.468,27.509,26.508,13.056,16.964,13.360,15.781,20.730,16.607,17.138,14.906,14.832,16.128,18.537,18.102,12.090,17.076,26.085,22.710,23.734,26.732,33.440,49.192,61.412,68.405,93.713,61.729,78.064,106.027,108.919,105.430,96.664,51.232,41.236,53.116,69.130,63.147,42.149,68.781,105.802,92.184,84.418,79.380,76.002,74.720,null
"Crude Oil (petroleum), West Texas Intermediate 40 API, Midland Texas, US$ per barrel",U.S. dollars,37.879,36.168,32.670,30.604,29.388,27.992,15.024,19.193,15.972,19.692,24.518,21.512,20.555,18.455,17.183,18.432,22.132,20.585,14.416,19.166,30.315,25.869,26.116,31.133,41.429,56.455,66.036,72.347,99.612,61.702,79.405,95.046,94.143,97.934,93.135,48.748,43.226,50.924,64.785,56.995,39.413,67.978,103.868,89.609,81.021,74.892,70.517,67.789,null
"Natural Gas, Russian Natural Gas border price in Germany, US$ per million metric British thermal units of gas",U.S. dollars,n/a,n/a,n/a,n/a,n/a,3.810,3.440,2.130,1.900,1.589,2.035,3.008,2.365,2.598,2.312,2.696,2.750,2.670,2.245,1.807,3.454,3.873,2.666,3.486,3.755,5.915,8.213,8.143,13.138,8.855,8.223,10.597,11.981,11.189,10.463,7.305,4.353,5.744,7.915,4.449,3.179,15.912,48.171,26.127,14.653,10.593,8.922,8.106,null
"Natural Gas, Indonesian Liquified Natural Gas in Japan, US$ per million metric British thermal units of liquid",U.S. dollars,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,3.744,3.598,3.285,3.557,3.903,3.761,2.754,3.343,5.235,4.669,4.427,4.981,5.891,7.033,8.049,8.411,11.591,7.491,9.381,15.553,18.145,17.339,16.998,10.958,7.439,7.247,9.795,5.445,4.366,18.600,44.883,27.048,27.048,27.048,27.048,27.048,null
"Natural Gas, Natural Gas spot price at the Henry Hub terminal in Louisiana, US$ per million metric British thermal units of gas",U.S. dollars,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,1.451,1.751,2.114,1.915,1.704,2.689,2.460,2.088,2.261,4.308,3.959,3.361,5.507,5.908,8.895,6.740,6.982,8.858,3.946,4.386,4.000,2.752,3.726,4.370,2.614,2.492,2.964,3.155,2.566,2.010,3.850,4.783,3.901,3.418,3.374,3.418,3.515,null
"Aluminum, 99.5% minimum purity, LME spot price, CIF UK ports, US$ per metric tonne",U.S. dollars,"1,774.913","1,262.730",991.568,"1,438.436","1,251.611","1,040.728","1,149.709","1,565.096","2,546.520","1,950.714","1,639.500","1,304.020","1,256.275","1,139.931","1,475.630","1,805.020","1,506.794","1,599.294","1,357.572","1,359.989","1,551.498","1,446.747","1,351.057","1,432.817","1,718.496","1,900.506","2,573.063","2,639.861","2,577.916","1,668.489","2,172.994","2,400.643","2,022.795","1,846.677","1,867.421","1,664.681","1,604.182","1,967.655","2,108.474","1,794.488","1,704.096","2,472.952","3,478.582","3,359.750","3,096.250","2,882.333","2,695.250","2,597.500",null
"Iron Ore, China import Iron Ore Fines 62% FE spot (CFR Tianjin port) US$ per metric ton",U.S. dollars,12.150,12.150,14.050,12.540,11.310,11.490,11.360,10.940,10.510,12.030,14.050,15.030,14.310,12.580,11.450,12.270,12.970,13.040,13.410,11.930,12.450,12.990,12.680,13.820,16.390,28.110,33.450,36.630,61.565,79.994,146.720,167.790,128.526,135.361,97.387,56.137,58.565,71.134,70.138,93.624,108.071,158.162,146.518,149.037,149.037,149.037,149.037,149.

In [0]:
#convert to pandas to apply transformation
pdSilver = dfSilver.toPandas()
pdSilver.head()

Out[11]:

,Subject_Descriptor,Units,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,Estimates_Start_After
0,"Crude Oil (petroleum), Dated Brent, light ble...",U.S. dollars,37.892,36.676,33.418,29.783,28.743,27.611,14.433,18.436,14.975,18.248,23.711,19.982,19.414,16.998,15.826,17.056,20.453,19.116,12.719,17.699,28.308,24.412,24.998,28.845,38.281,54.433,65.376,72.731,97.569,61.865,79.635,111.082,111.960,108.844,98.943,52.399,44.048,54.401,71.071,64.032,42.298,70.443,110.815,96.092,87.238,81.363,77.583,74.923,None
1,"Oil; Dubai, medium, Fateh 32 API, fob Dubai Cr...",U.S. dollars,35.846,34.294,31.757,28.468,27.509,26.508,13.056,16.964,13.360,15.781,20.730,16.607,17.138,14.906,14.832,16.128,18.537,18.102,12.090,17.076,26.085,22.710,23.734,26.732,33.440,49.192,61.412,68.405,93.713,61.729,78.064,106.027,108.919,105.430,96.664,51.232,41.236,53.116,69.130,63.147,42.149,68.781,105.802,92.184,84.418,79.380,76.002,74.720,None
2,"Crude Oil (petroleum), West Texas Intermediate...",U.S. dollars,37.879,36.168,32.670,30.604,29.388,27.992,15.024,19.193,15.972,19.692,24.518,21.512,20.555,18.455,17.183,18.432,22.132,20.585,14.416,19.166,30.315,25.869,26.116,31.133,41.429,56.455,66.036,72.347,99.612,61.702,79.405,95.046,94.143,97.934,93.135,48.748,43.226,50.924,64.785,56.995,39.413,67.978,103.868,89.609,81.021,74.892,70.517,67.789,None
3,"Natural Gas, Russian Natural Gas border price ...",U.S. dollars,n/a,n/a,n/a,n/a,n/a,3.810,3.440,2.130,1.900,1.589,2.035,3.008,2.365,2.598,2.312,2.696,2.750,2.670,2.245,1.807,3.454,3.873,2.666,3.486,3.755,5.915,8.213,8.143,13.138,8.855,8.223,10.597,11.981,11.189,10.463,7.305,4.353,5.744,7.915,4.449,3.179,15.912,48.171,26.127,14.653,10.593,8.922,8.106,None
4,"Natural Gas, Indonesian Liquified Natural Gas ...",U.S. dollars,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,n/a,3.744,3.598,3.285,3.557,3.903,3.761,2.754,3.343,5.235,4.669,4.427,4.981,5.891,7.033,8.049,8.411,11.591,7.491,9.381,15.553,18.145,17.339,16.998,10.958,7.439,7.247,9.795,5.445,4.366,18.600,44.883,27.048,27.048,27.048,27.048,27.048,None


In [0]:
#replace n/a strings to None
pdSilver = pdSilver.replace({'n/a': None})

In [0]:

#convert date columns to rows (transpose)

pdSilver = pdSilver.melt(id_vars=["Subject_Descriptor"
,"Units"
,"Estimates_Start_After"], 
        var_name="period", 
        value_name="value")




In [0]:
import pandas as pd
#convert date(year) values to int type 
pdSilver['period'] = pdSilver['period'].astype('int')
pdSilver.dtypes

Out[14]: Subject_Descriptor object
Units object
Estimates_Start_After object
period int64
value object
dtype: object

Rename column names to match Interface Contract requirements (standard names)

In [0]:
#rename columns

pdSilver.rename(columns={ 'Subject_Descriptor': 'subject_descriptor', 'Units': 'unitMeasure_code', 'Estimates_Start_After': 'dateEstimatedStartAfter'}, inplace=True)

In [0]:
#reset index once applied transformation
pdSilver.reset_index(drop=True, inplace=True)

We can check the transformations are being correctly applied.

In [0]:
print(pdSilver.count())
pdSilver.head(10)

subject_descriptor 384
unitMeasure_code 384
dateEstimatedStartAfter 0
period 384
value 356
dtype: int64
Out[17]:

,subject_descriptor,unitMeasure_code,dateEstimatedStartAfter,period,value
0,"Crude Oil (petroleum), Dated Brent, light ble...",U.S. dollars,None,1980,37.892
1,"Oil; Dubai, medium, Fateh 32 API, fob Dubai Cr...",U.S. dollars,None,1980,35.846
2,"Crude Oil (petroleum), West Texas Intermediate...",U.S. dollars,None,1980,37.879
3,"Natural Gas, Russian Natural Gas border price ...",U.S. dollars,None,1980,None
4,"Natural Gas, Indonesian Liquified Natural Gas ...",U.S. dollars,None,1980,None
5,"Natural Gas, Natural Gas spot price at the Hen...",U.S. dollars,None,1980,None
6,"Aluminum, 99.5% minimum purity, LME spot price...",U.S. dollars,None,1980,"1,774.913"
7,"Iron Ore, China import Iron Ore Fines 62% FE s...",U.S. dollars,None,1980,12.150
8,"Crude Oil (petroleum), Dated Brent, light ble...",U.S. dollars,None,1981,36.676
9,"Oil; Dubai, medium, Fateh 32 API, fob Dubai Cr...",U.S. dollars,None,1981,34.294


In [0]:
#create new column name commodities which contains a short name of subject_descriptor

def label_race(pdSilver):
    pdSilver.loc[pdSilver.subject_descriptor.astype(str).str.startswith('Natural Gas, Russian Natural Gas'),'commodities'] = 'Russian Natural Gas'
    pdSilver.loc[pdSilver.subject_descriptor.astype(str).str.startswith('Natural Gas, Indonesian Liquified'),'commodities'] = 'Indonesian Natural Gas Liquified'
    pdSilver.loc[pdSilver.subject_descriptor.astype(str).str.startswith('Natural Gas, Natural Gas spot price at the Henry'),'commodities'] = 'USA Natural Gas'
    pdSilver.loc[pdSilver.subject_descriptor.astype(str).str.startswith('Crude Oil (petroleum),  Dated Brent, light blend 38 API, fob U.K., US$ per barrel'),'commodities'] = 'Crude Oil Dated Brent per barrel'
    pdSilver.loc[pdSilver.subject_descriptor.astype(str).str.startswith('Oil; Dubai, medium, Fateh 32 API,'),'commodities'] = 'Crude oil Dubai Fateh per barrel'
    pdSilver.loc[pdSilver.subject_descriptor.astype(str).str.startswith('Crude Oil (petroleum), West Texas'),'commodities'] = 'Crude oil West Texas per barrel'
    pdSilver.loc[pdSilver.subject_descriptor.astype(str).str.startswith('Aluminum, 99.5%'),'commodities'] = 'Aluminium Prices per metric tonne'
    pdSilver.loc[pdSilver.subject_descriptor.astype(str).str.startswith('Iron Ore, China'),'commodities'] = 'Iron ore prices (China Import) per metric ton'



In [0]:
label_race(pdSilver)

In [0]:
display(pdSilver)

subject_descriptor,unitMeasure_code,dateEstimatedStartAfter,period,value,commodities
"Crude Oil (petroleum), Dated Brent, light blend 38 API, fob U.K., US$ per barrel",U.S. dollars,null,1980,37.892,Crude Oil Dated Brent per barrel
"Oil; Dubai, medium, Fateh 32 API, fob Dubai Crude Oil (petroleum), Dubai Fateh Fateh 32 API, US$ per barrel",U.S. dollars,null,1980,35.846,Crude oil Dubai Fateh per barrel
"Crude Oil (petroleum), West Texas Intermediate 40 API, Midland Texas, US$ per barrel",U.S. dollars,null,1980,37.879,Crude oil West Texas per barrel
"Natural Gas, Russian Natural Gas border price in Germany, US$ per million metric British thermal units of gas",U.S. dollars,null,1980,null,Russian Natural Gas
"Natural Gas, Indonesian Liquified Natural Gas in Japan, US$ per million metric British thermal units of liquid",U.S. dollars,null,1980,null,Indonesian Natural Gas Liquified
"Natural Gas, Natural Gas spot price at the Henry Hub terminal in Louisiana, US$ per million metric British thermal units of gas",U.S. dollars,null,1980,null,USA Natural Gas
"Aluminum, 99.5% minimum purity, LME spot price, CIF UK ports, US$ per metric tonne",U.S. dollars,null,1980,"1,774.913",Aluminium Prices per metric tonne
"Iron Ore, China import Iron Ore Fines 62% FE spot (CFR Tianjin port) US$ per metric ton",U.S. dollars,null,1980,12.150,Iron ore prices (China Import) per metric ton
"Crude Oil (petroleum), Dated Brent, light blend 38 API, fob U.K., US$ per barrel",U.S. dollars,null,1981,36.676,Crude Oil Dated Brent per barrel
"Oil; Dubai, medium, Fateh 32 API, fob Dubai Crude Oil (petroleum), Dubai Fateh Fateh 32 API, US$ per barrel",U.S. dollars,null,1981,34.294,Crude oil Dubai Fateh per barrel


In [0]:
#revert back to spark Dataframe so we can store data in delta format
dfSilver = spark.createDataFrame(pdSilver)

dfSilver.printSchema()



root
-- subject_descriptor: string (nullable = true)
-- unitMeasure_code: string (nullable = true)
-- dateEstimatedStartAfter: void (nullable = true)
-- period: long (nullable = true)
-- value: string (nullable = true)
-- commodities: string (nullable = true)

In [0]:
dfSilver = dfSilver.dropDuplicates()

dfSilver.show()
dfSilver.count()

+--------------------+----------------+-----------------------+------+---------+--------------------+
 subject_descriptor|unitMeasure_code|dateEstimatedStartAfter|period| value| commodities|
+--------------------+----------------+-----------------------+------+---------+--------------------+
Crude Oil (petrol...| U.S. dollars| null| 1980| 37.879|Crude oil West Te...|
Natural Gas, Indo...| U.S. dollars| null| 1980| null|Indonesian Natura...|
Crude Oil (petrol...| U.S. dollars| null| 1981| 36.676|Crude Oil Dated B...|
Aluminum, 99.5% m...| U.S. dollars| null| 1980|1,774.913|Aluminium Prices ...|
Iron Ore, China i...| U.S. dollars| null| 1980| 12.150|Iron ore prices (...|
Oil; Dubai, mediu...| U.S. dollars| null| 1981| 34.294|Crude oil Dubai F...|
Oil; Dubai, mediu...| U.S. dollars| null| 1982| 31.757|Crude oil Dubai F...|
Natural Gas, Indo...| U.S. dollars| null| 1982| null|Indonesian Natura...|
Crude Oil (petrol...| U.S. dollars| null| 1982| 33.418|Crude Oil Dated B...|
Crude Oil (petrol...| U.S. dollars| null| 1981| 36.168|Crude oil West Te...|
Natural Gas, Russ...| U.S. dollars| null| 1982| null| Russian Natural Gas|
Natural Gas, Russ...| U.S. dollars| null| 1980| null| Russian Natural Gas|
Aluminum, 99.5% m...| U.S. dollars| null| 1981|1,262.730|Aluminium Prices ...|
Natural Gas, Natu...| U.S. dollars| null| 1980| null| USA Natural Gas|
Crude Oil (petrol...| U.S. dollars| null| 1980| 37.892|Crude Oil Dated B...|
Natural Gas, Russ...| U.S. dollars| null| 1981| null| Russian Natural Gas|
Natural Gas, Natu...| U.S. dollars| null| 1981| null| USA Natural Gas|
Crude Oil (petrol...| U.S. dollars| null| 1982| 32.670|Crude oil West Te...|
Natural Gas, Indo...| U.S. dollars| null| 1981| null|Indonesian Natura...|
Oil; Dubai, mediu...| U.S. dollars| null| 1980| 35.846|Crude oil Dubai F...|
+--------------------+----------------+-----------------------+------+---------+--------------------+
only showing top 20 rows

Out[25]: 384

We can save now the cleaned tables in the Silver stage

In [0]:
#save cleaned data in its silver table
dfSilver.write.format('delta').mode('overwrite').save(actualAndForecastCommoditiesPrices_Silver)



In [0]:
#Load and verify that it's been saved properly
commodities = spark.read.format("delta").load(actualAndForecastCommoditiesPrices_Silver)

display(commodities)

subject_descriptor,unitMeasure_code,period,value,commodities
"Crude Oil (petroleum), West Texas Intermediate 40 API, Midland Texas, US$ per barrel",U.S. dollars,1983,30.604,Crude oil West Texas per barrel
"Oil; Dubai, medium, Fateh 32 API, fob Dubai Crude Oil (petroleum), Dubai Fateh Fateh 32 API, US$ per barrel",U.S. dollars,1991,16.607,Crude oil Dubai Fateh per barrel
"Natural Gas, Natural Gas spot price at the Henry Hub terminal in Louisiana, US$ per million metric British thermal units of gas",U.S. dollars,1986,null,USA Natural Gas
"Natural Gas, Russian Natural Gas border price in Germany, US$ per million metric British thermal units of gas",U.S. dollars,1988,1.900,Russian Natural Gas
"Crude Oil (petroleum), West Texas Intermediate 40 API, Midland Texas, US$ per barrel",U.S. dollars,1980,37.879,Crude oil West Texas per barrel
"Natural Gas, Indonesian Liquified Natural Gas in Japan, US$ per million metric British thermal units of liquid",U.S. dollars,1980,null,Indonesian Natural Gas Liquified
"Aluminum, 99.5% minimum purity, LME spot price, CIF UK ports, US$ per metric tonne",U.S. dollars,1988,"2,546.520",Aluminium Prices per metric tonne
"Natural Gas, Natural Gas spot price at the Henry Hub terminal in Louisiana, US$ per million metric British thermal units of gas",U.S. dollars,1983,null,USA Natural Gas
"Natural Gas, Natural Gas spot price at the Henry Hub terminal in Louisiana, US$ per million metric British thermal units of gas",U.S. dollars,1989,null,USA Natural Gas
"Aluminum, 99.5% minimum purity, LME spot price, CIF UK ports, US$ per metric tonne",U.S. dollars,1983,"1,438.436",Aluminium Prices per metric tonne
